In [484]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,KFold,ShuffleSplit,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import r2_score,roc_auc_score,accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
from mlxtend.classifier import StackingCVClassifier

%matplotlib inline
import seaborn as sns
import xgboost as xgb
from scipy.stats import pearsonr

In [50]:
pd.set_option('max_columns',100)

In [319]:
train=pd.read_csv('/Users/shashank/Downloads/Societe2/train.csv')
test=pd.read_csv('/Users/shashank/Downloads/Societe2/test.csv')
sample=pd.read_csv('/Users/shashank/Downloads/Societe2/sample_submissions.csv')

In [320]:
# ids=test['transaction_id']
# train.drop(['transaction_id'],axis=1,inplace=True)
# test.drop(['transaction_id'],axis=1,inplace=True)

In [321]:
train['cat_var_1'].fillna('gf',inplace=True)
test['cat_var_1'].fillna('gf',inplace=True)
train['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_6'].fillna('zs',inplace=True)
train['cat_var_8'].fillna('dn',inplace=True)
test['cat_var_8'].fillna('dn',inplace=True)

In [322]:
# ## merge 19 and 20
# train.drop(['cat_var_20'],axis=1,inplace=True)
# test.drop(['cat_var_20'],axis=1,inplace=True)

In [323]:
# ## merge 21,22,23
# train['cat_var_21-23']=train.loc[:,['cat_var_22','cat_var_23']].sum(axis=1)
# test['cat_var_21-23']=test.loc[:,['cat_var_22','cat_var_23']].sum(axis=1)

# train.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)
# test.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)

In [324]:
# ## merge 24-42
# train['cat_var_24-42']=train.loc[:,list(train.columns[27:45])].sum(axis=1)
# test['cat_var_24-42']=test.loc[:,list(test.columns[27:45])].sum(axis=1)

# train.drop(list(train.columns[27:45]),axis=1,inplace=True)
# test.drop(list(test.columns[27:45]),axis=1,inplace=True)

In [325]:
# train.drop(['cat_var_24'],axis=1,inplace=True)
# test.drop(['cat_var_24'],axis=1,inplace=True)

In [326]:
# li=[]
# for index,row in test.iterrows():
#     if (row['cat_var_1'] in ['jk', 'gz', 'yw']) or (row['cat_var_3'] in ['ex', 'ye', 'hr', 'ap']) or (row['cat_var_8'] in ['da', 'db', 'mh']):
#         li.append(True)
#     else: li.append(False)

In [230]:
# dic=dict(df['cat_var_21-23'].value_counts())
# df['var_21-23_counts']=df['cat_var_21-23'].apply(lambda x: dic[x])
# df.drop(['cat_var_21-23'],axis=1,inplace=True)

In [231]:
# dic=dict(df['cat_var_24-42'].value_counts())
# df['var_24-42_counts']=df['cat_var_24-42'].apply(lambda x: dic[x])
# df.drop(['cat_var_24-42'],axis=1,inplace=True)

In [ ]:
df=pd.concat([train,test])
for i in range(1,19):
    if i not in [1,3,6,8]:
        dic=dict(df['cat_var_%d'%i].value_counts())
        df['var_%d_counts'%i]=df['cat_var_%d'%i].apply(lambda x: dic[x])
        df.drop(['cat_var_%d'%i],axis=1,inplace=True)

In [328]:
train=df.iloc[:len(train),:]
test=df.iloc[len(train):,:]

In [466]:
def to_popularity(col):
    df=pd.concat([train,test])
    dic=dict(df[col].value_counts())
    df['popularity_%s'%col]=df[col].apply(lambda x: dic[x])
    df.drop([col],axis=1,inplace=True)
    train=df.iloc[:len(train),:]
    test=df.iloc[len(train):,:]
    

In [329]:
print len(train),len(test)

348978 523466


In [331]:
trainX=train.copy(deep=True)
testX=test.copy(deep=True)

In [332]:
trainX.drop(['var_8_counts'],axis=1,inplace=True)
testX.drop(['var_8_counts'],axis=1,inplace=True)

In [333]:
predictors=[x for x in trainX.columns if x not in ['target','transaction_id']]
target=['target']

In [467]:
X_train,X_test,y_train,y_test=train_test_split(trainX[predictors].values,trainX[target].values,test_size=0.3)

In [460]:
def run(X,y,test,params):
    dtrain=xgb.DMatrix(X,label=y)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(params,dtrain,100)
    preds=bst.predict(dtest)
    return bst,preds

params={
    'objective':'binary:logistic',
    'eta':0.11,
    'max_depth':9,
#     'min_child_weight':7,
#     'gamma':0.1,
#     'subsample':0.9,
#     'colsample_bytree':0.7,
#     'scale_pos_weight':1,
#     'reg_alpha':1e-05
}

In [485]:
xg=xgb.XGBClassifier(max_depth=9,n_estimators=99,learning_rate=0.11)
# xg.fit(X_train,y_train)
# xgp=xg.predict_proba(X_test)
# xgp=xgp[:,1]
# xgp1=(xgp>0.5).astype('int')
# print roc_auc_score(y_test,xgp)
# print accuracy_score(y_test,xgp1)
# print confusion_matrix(y_test,xgp1)

In [474]:
pearsonr(rfp,xgp)

(0.89024277599770585, 0.0)

In [486]:
rf=RandomForestClassifier(max_depth=9,n_estimators=89)
# rf.fit(X_train,y_train)
# rfp=rf.predict_proba(X_test)
# rfp=rfp[:,1]
# rfp1=(rfp>0.5).astype('int')
# print roc_auc_score(y_test,rfp)
# print accuracy_score(y_test,rfp1)
# print confusion_matrix(y_test,rfp1)

In [469]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
lrp=lr.predict_proba(X_test)
lrp=lrp[:,1]
lrp1=(lrp>0.5).astype('int')
print roc_auc_score(y_test,lrp)
print accuracy_score(y_test,lrp1)
print confusion_matrix(y_test,lrp1)

/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.658681219015
0.90851433702
[[92942   393]
 [ 9185  2174]]


In [471]:
bst1,p1=run(X_train,y_train,X_test,params)
print roc_auc_score(y_test,p1)
p2=(p1>0.5).astype('int')
print accuracy_score(y_test,p2)
print confusion_matrix(y_test,p2)

0.72785728681
0.932250176705
[[93210   125]
 [ 6968  4391]]


In [462]:
# bst1,p1=run(X_train,y_train,X_test,params)
# print roc_auc_score(y_test,p1)
# p2=(p1>0.5).astype('int')
# print accuracy_score(y_test,p2)
# print confusion_matrix(y_test,p2)

In [463]:
bst,xgpreds=run(trainX[predictors].values,trainX[target].values,testX[predictors].values,params)

In [464]:
subs=pd.DataFrame({
    'transaction_id':ids,
    'target':xgpreds
},columns=['transaction_id','target'])

In [465]:
subs.to_csv('/Users/shashank/Downloads/Societe2/72.csv',index=False)

In [361]:
high=pd.read_csv('/Users/shashank/Downloads/7363.csv')
second=pd.read_csv('/Users/shashank/Downloads/Societe2/45.csv')

In [372]:
f=0.6*high['target']+0.4*second['target']

In [311]:
normal=pd.DataFrame({'transaction_id':test1['transaction_id'],'target':xgpreds},columns=['transaction_id','target'])

In [312]:
log=pd.read_csv('/Users/shashank/Downloads/log1.csv')

In [313]:
log=log.set_index('Unnamed: 0')

In [314]:
f=pd.concat([normal,log])

In [315]:
f=f.sort_index()

In [318]:
f.to_csv('/Users/shashank/Downloads/Societe2/combined.csv',index=False)

In [369]:
# ---45 second high --73629
# ---47 highest -- 0.73683	ensemble of 45 and 73630
## -- 67 -- 0.73653 with eta 0.11 depth 9 and trees=99
## --68 120 tress 73594
### 69 -- 0.73602 108
### 70 -- 0.73609 104 trees
### 71 --- 0.73640 102 trees
### 72 --- 0.73650 100 trees

In [495]:
X=trainX[predictors]
Y=trainX[target]

In [496]:
print X.shape,Y.shape

(348978, 48) (348978, 1)


In [508]:
knn=KNeighborsClassifier(n_neighbors=3)
ex=ExtraTreesClassifier(max_depth=9,n_estimators=99)

In [509]:
sclf = StackingCVClassifier(classifiers=[ xg,rf,knn,ex], 
                            meta_classifier=xg)

In [510]:
scores = cross_val_score(sclf, X.values, Y.values.ravel(), cv=3, scoring='roc_auc')

In [511]:
scores.mean()

0.69931426307776434

In [507]:
scores.mean()

0.63352823033963468

In [501]:
scores.mean()

0.69325697441161493

In [504]:
scores.mean()

0.69325282772709818

In [490]:
for clf, label in zip([xg, rf, sclf], 
                      ['xg', 
                       'Random Forest', 
                       'StackingClassifier']):

    scores = cross_val_score(clf, X, Y, 
                                              cv=3, scoring='roc_auc')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))


Accuracy: 0.73 (+/- 0.00) [xg]


/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


Accuracy: 0.72 (+/- 0.00) [Random Forest]


/anaconda2/lib/python2.7/site-packages/mlxtend/classifier/stacking_cv_classification.py:175: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X[train_index], y[train_index])


ValueError: all the input arrays must have same number of dimensions